# OSMAN BUKARI - PROFESSIONAL SCOUTING DASHBOARD
## Portland Timbers #10 | Left Midfielder | Age 26
### Four-Pillar Evaluation: Technical | Physical | Tactical | Psychological

---

**Report Date:** January 29, 2026  
**Matches Analyzed:** 25  
**Data Source:** MLS 2025 Season  
**Analyst:** Jacob Farrington

## 📦 SETUP: Import Libraries

In [31]:
# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

# Visualization
import plotly.io as pio
pio.renderers.default = "notebook"
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Plotly default template
import plotly.io as pio
pio.templates.default = "plotly_white"

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## 📥 LOAD DATA

Load all sheets from the Excel file we created.

In [32]:
# ====================================================================
# PLAYER CONFIGURATION - Change this to switch between players
# ====================================================================

# Available players: 'DaCosta', 'Bukari', 'Arfsten'
PLAYER = 'Bukari'  # <-- CHANGE THIS TO SWITCH PLAYERS

# Player file mapping
player_files = {
    'DaCosta': 'data/DaCosta.xlsx',
    'Bukari': 'data/Bukari.xlsx',
    'Arfsten': 'data/Arfsten.xlsx'
}

# Player display names
player_names = {
    'DaCosta': '{player_display_name}',
    'Bukari': 'Osman Bukari',
    'Arfsten': 'Max Arfsten'
}

# Get the file path and display name
file_name = player_files[PLAYER]
player_display_name = player_names[PLAYER]

'''print(f"✅ Loading data for: {player_display_name}")
print(f"   File: {file_name}")'''

# ====================================================================
# LOAD DATA FROM EXCEL FILE
# ====================================================================

file_path = file_name

# Load all sheets
player_info = pd.read_excel(file_path, sheet_name='Player_Info')
match_data = pd.read_excel(file_path, sheet_name='Match_Data')
technical = pd.read_excel(file_path, sheet_name='Technical')
physical = pd.read_excel(file_path, sheet_name='Physical')
tactical = pd.read_excel(file_path, sheet_name='Tactical')
psychological = pd.read_excel(file_path, sheet_name='Psychological')
benchmarks = pd.read_excel(file_path, sheet_name='Benchmarks')

# Merge all data into single dataframe (df) using Match_ID
df = match_data.copy()
df = df.merge(technical, on='Match_ID', how='left')
df = df.merge(physical, on='Match_ID', how='left')
df = df.merge(tactical, on='Match_ID', how='left')
df = df.merge(psychological, on='Match_ID', how='left')

# Convert Date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

print("✅ Data loaded successfully!")
print(f"\n📊 Dataset Shape: {len(df)} matches x {len(df.columns)} columns")
print(f"\n📋 Player Info:")
for idx, row in player_info.iterrows():
    print(f"   {row['Attribute']}: {row['Value']}")


✅ Data loaded successfully!

📊 Dataset Shape: 25 matches x 84 columns

📋 Player Info:
   Name: Osman Bukari
   Number: 11
   Club: Austin FC
   Primary_Position: Right Winger
   Secondary_Positions: LW, RW, RM, LM
   Age: 25
   Height_CM: 173
   Weight_KG: 68
   Nationality: Ghanaian
   Preferred_Foot: Right
   Contract_Expires: 2027-12-31
   Market_Value_USD: 6500000
   Previous_Club: Red Star Belgrade


## 🧮 MEASURES: Implement DAX-style Calculations

These are Python equivalents of the Power BI DAX measures.

In [33]:
class ScoutingMetrics:
    """Calculate all scouting metrics (equivalent to Power BI DAX measures)"""
    
    def __init__(self, df):
        self.df = df
        # Removed self.total_minutes to avoid conflict with total_minutes() method
        
    # ==========================================
    # GENERAL MEASURES
    # ==========================================
    
    def total_matches(self):
        return len(self.df)
    
    def total_minutes(self):
        return self.df['Minutes'].sum()
    
    def avg_minutes(self):
        return self.df['Minutes'].mean()
    
    def matches_started(self):
        return (self.df['Started'] == 1).sum()
    
    def win_rate(self):
        wins = (self.df['Result'] == 'W').sum()
        return (wins / len(self.df)) * 100
    
    # ==========================================
    # TECHNICAL MEASURES
    # ==========================================
    
    def total_goals(self):
        return self.df['Goals'].sum()
    
    def total_assists(self):
        return self.df['Assists'].sum()
    
    def goals_per_90(self):
        return (self.df['Goals'].sum() * 90) / self.total_minutes()
    
    def assists_per_90(self):
        return (self.df['Assists'].sum() * 90) / self.total_minutes()
    
    def avg_pass_accuracy(self):
        return self.df['Pass_Accuracy'].mean()
    
    def avg_dribble_success(self):
        return self.df['Dribble_Success_Rate'].mean()
    
    def avg_shot_accuracy(self):
        return self.df['Shot_Accuracy'].mean()
    
    def total_xg(self):
        return self.df['xG'].sum()
    
    def xg_per_90(self):
        return (self.df['xG'].sum() * 90) / self.total_minutes()
    
    def total_shots(self):
        return self.df['Shots_Total'].sum()
    
    def shots_per_90(self):
        return (self.df['Shots_Total'].sum() * 90) / self.total_minutes()
    
    def keypasses_per_90(self):
        return (self.df['Key_Passes'].sum() * 90) / self.total_minutes()
    
    def avg_first_touch_rating(self):
        return self.df['First_Touch_Rating'].mean()
    
    def technical_rating(self):
        return (
            (self.avg_pass_accuracy() / 100) * 0.30 +
            (self.avg_dribble_success() / 100) * 0.25 +
            (self.avg_shot_accuracy() / 100) * 0.15 +
            (self.avg_first_touch_rating() / 10) * 0.30
        ) * 10
    
    # ==========================================
    # PHYSICAL MEASURES
    # ==========================================
    
    def distance_per_90(self):
        return (self.df['Distance_Covered_KM'].sum() * 90) / self.total_minutes()
    
    def avg_top_speed(self):
        return self.df['Top_Speed_KMH'].mean()
    
    def sprints_per_90(self):
        return (self.df['Sprints'].sum() * 90) / self.total_minutes()
    
    def hi_runs_per_90(self):
        return (self.df['High_Intensity_Runs'].sum() * 90) / self.total_minutes()
    
    def avg_duel_success(self):
        return self.df['Duel_Success_Rate'].mean()
    
    def avg_aerial_success(self):
        return self.df['Aerial_Success_Rate'].mean()
    
    def avg_stamina_indicator(self):
        return self.df['Stamina_Indicator'].mean()
    
    def avg_strength_rating(self):
        return self.df['Strength_Rating'].mean()
    
    def physical_rating(self):
        return (
            (self.distance_per_90() / 12) * 0.30 +
            (self.hi_runs_per_90() / 80) * 0.25 +
            (self.avg_duel_success() / 100) * 0.20 +
            (self.avg_strength_rating() / 10) * 0.15 +
            (self.avg_top_speed() / 35) * 0.10
        ) * 10
    
    # ==========================================
    # TACTICAL MEASURES
    # ==========================================
    
    def tackles_per_90(self):
        return (self.df['Tackles_Won'].sum() * 90) / self.total_minutes()
    
    def interceptions_per_90(self):
        return (self.df['Interceptions'].sum() * 90) / self.total_minutes()
    
    def defensive_actions_per_90(self):
        defensive_actions = (self.df['Tackles_Won'] + self.df['Interceptions'] + 
                           self.df['Clearances'] + self.df['Blocks']).sum()
        return (defensive_actions * 90) / self.total_minutes()
    
    def pressures_per_90(self):
        return (self.df['Pressures_Applied'].sum() * 90) / self.total_minutes()
    
    def avg_pressure_success(self):
        total_pressures = self.df['Pressures_Applied'].sum()
        successful = self.df['Successful_Pressures'].sum()
        return (successful / total_pressures * 100) if total_pressures > 0 else 0
    
    def avg_positioning_rating(self):
        return self.df['Positioning_Rating'].mean()
    
    def avg_decision_making(self):
        return self.df['Decision_Making_Rating'].mean()
    
    def avg_defensive_work_rate(self):
        return self.df['Defensive_Work_Rate'].mean()
    
    def avg_attacking_positioning(self):
        return self.df['Attacking_Positioning'].mean()
    
    def tactical_rating(self):
        return (
            (self.avg_positioning_rating() / 10) * 0.25 +
            (self.avg_decision_making() / 10) * 0.25 +
            (self.avg_defensive_work_rate() / 10) * 0.20 +
            (self.avg_attacking_positioning() / 10) * 0.20 +
            (self.defensive_actions_per_90() / 5) * 0.10
        ) * 10
    
    # ==========================================
    # PSYCHOLOGICAL MEASURES
    # ==========================================
    
    def avg_performance_rating(self):
        return self.df['Performance_Rating'].mean()
    
    def avg_pressure_performance(self):
        return self.df['Pressure_Performance_Rating'].mean()
    
    def performance_consistency(self):
        return 10 - self.df['Performance_Rating'].std()
    
    def total_yellow_cards(self):
        return self.df['Yellow_Cards'].sum()
    
    def total_red_cards(self):
        return self.df['Red_Cards'].sum()
    
    def fouls_per_90(self):
        return (self.df['Fouls_Committed'].sum() * 90) / self.total_minutes()
    
    def avg_composure(self):
        return self.df['Composure_Rating'].mean()
    
    def avg_confidence(self):
        return self.df['Confidence_Level'].mean()
    
    def avg_mental_strength(self):
        return self.df['Mental_Strength'].mean()
    
    def discipline_rating(self):
        return max(0, 10 - (self.fouls_per_90() * 2) - (self.total_yellow_cards() * 0.5))
    
    def psychological_rating(self):
        return (
            (self.avg_performance_rating() / 10) * 0.25 +
            (self.performance_consistency() / 10) * 0.20 +
            (self.discipline_rating() / 10) * 0.20 +
            (self.avg_composure() / 10) * 0.15 +
            (self.avg_confidence() / 10) * 0.10 +
            (self.avg_mental_strength() / 10) * 0.10
        ) * 10
    
    # ==========================================
    # OVERALL RATING
    # ==========================================
    
    def overall_rating(self):
        return (
            self.technical_rating() * 0.30 +
            self.physical_rating() * 0.25 +
            self.tactical_rating() * 0.25 +
            self.psychological_rating() * 0.20
        )
    
    def get_all_metrics(self):
        """Return dictionary of all metrics"""
        return {
            # General
            'Total Matches': self.total_matches(),
            'Total Minutes': self.total_minutes(),
            'Avg Minutes': self.avg_minutes(),
            'Matches Started': self.matches_started(),
            'Win Rate %': self.win_rate(),
            
            # Technical
            'Goals': self.total_goals(),
            'Assists': self.total_assists(),
            'Goals per 90': self.goals_per_90(),
            'Assists per 90': self.assists_per_90(),
            'Pass Accuracy %': self.avg_pass_accuracy(),
            'Dribble Success %': self.avg_dribble_success(),
            'Shot Accuracy %': self.avg_shot_accuracy(),
            'xG': self.total_xg(),
            'xG per 90': self.xg_per_90(),
            'Shots per 90': self.shots_per_90(),
            'Key Passes per 90': self.keypasses_per_90(),
            'Technical Rating': self.technical_rating(),
            
            # Physical
            'Distance per 90 (km)': self.distance_per_90(),
            'Top Speed (km/h)': self.avg_top_speed(),
            'Sprints per 90': self.sprints_per_90(),
            'HI Runs per 90': self.hi_runs_per_90(),
            'Duel Success %': self.avg_duel_success(),
            'Aerial Success %': self.avg_aerial_success(),
            'Physical Rating': self.physical_rating(),
            
            # Tactical
            'Tackles per 90': self.tackles_per_90(),
            'Interceptions per 90': self.interceptions_per_90(),
            'Defensive Actions per 90': self.defensive_actions_per_90(),
            'Pressures per 90': self.pressures_per_90(),
            'Pressure Success %': self.avg_pressure_success(),
            'Tactical Rating': self.tactical_rating(),
            
            # Psychological
            'Avg Performance Rating': self.avg_performance_rating(),
            'Performance Consistency': self.performance_consistency(),
            'Yellow Cards': self.total_yellow_cards(),
            'Red Cards': self.total_red_cards(),
            'Fouls per 90': self.fouls_per_90(),
            'Discipline Rating': self.discipline_rating(),
            'Psychological Rating': self.psychological_rating(),
            
            # Overall
            'Overall Rating': self.overall_rating()
        }

# Initialize metrics calculator
metrics = ScoutingMetrics(df)

# Get all metrics
all_metrics = metrics.get_all_metrics()

print("✅ Metrics calculated successfully!")
print(f"\n📊 KEY METRICS:")
print(f"   Overall Rating: {all_metrics['Overall Rating']:.1f}/10")
print(f"   Technical:      {all_metrics['Technical Rating']:.1f}/10")
print(f"   Physical:       {all_metrics['Physical Rating']:.1f}/10")
print(f"   Tactical:       {all_metrics['Tactical Rating']:.1f}/10")
print(f"   Psychological:  {all_metrics['Psychological Rating']:.1f}/10")


✅ Metrics calculated successfully!

📊 KEY METRICS:
   Overall Rating: 6.6/10
   Technical:      6.6/10
   Physical:       7.4/10
   Tactical:       5.8/10
   Psychological:  6.8/10


---

# PAGE 1: EXECUTIVE SUMMARY

## Overview Dashboard with Key Ratings

In [47]:
# Create executive summary dashboard
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Overall Rating', 'Four Pillar Ratings', 'Team Results',
                    'Key Stats', 'Goals & Assists Timeline', 'Position Distribution'),
    specs=[[{'type': 'indicator'}, {'type': 'bar'}, {'type': 'pie'}],
           [{'type': 'table', 'colspan': 3}, None, None],
           [{'type': 'scatter', 'colspan': 2}, None, {'type': 'bar'}]],
    row_heights=[0.35, 0.25, 0.40],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# 1. Overall Rating Gauge
fig.add_trace(go.Indicator(
    mode = "gauge+number+delta",
    value = all_metrics['Overall Rating'],
    title = {'text': "", 'font': {'size': 20}},
    delta = {'reference': 7.5},
    gauge = {
        'axis': {'range': [None, 10], 'tickwidth': 1, 'tickcolor': "darkblue"},
        'bar': {'color': "#00A86B"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 6.5], 'color': '#FFE5E5'},
            {'range': [6.5, 7.5], 'color': '#FFF5E5'},
            {'range': [7.5, 10], 'color': '#E5FFE5'}],
        'threshold': {
            'line': {'color': "red", 'width': 4},
            'thickness': 0.75,
            'value': 7.5}
    }
), row=1, col=1)

# 2. Four Pillar Ratings Bar Chart
pillars = ['Technical', 'Physical', 'Tactical', 'Psychological']
ratings = [
    all_metrics['Technical Rating'],
    all_metrics['Physical Rating'],
    all_metrics['Tactical Rating'],
    all_metrics['Psychological Rating']
]
colors = ['#00A86B' if r >= 7.5 else '#FFB81C' if r >= 6.5 else '#D32F2F' for r in ratings]

fig.add_trace(go.Bar(
    x=pillars,
    y=ratings,
    marker_color=colors,
    text=[f'{r:.1f}' for r in ratings],
    textposition='outside',
    showlegend=False
), row=1, col=2)

# 3. Team Results Pie Chart
result_counts = df['Result'].value_counts()
fig.add_trace(go.Pie(
    labels=['Wins', 'Draws', 'Losses'],
    values=[result_counts.get('W', 0), result_counts.get('D', 0), result_counts.get('L', 0)],
    marker_colors=['#00A86B', '#FFB81C', '#D32F2F'],
    hole=0.4
), row=1, col=3)

# 4. Key Stats Table
key_stats = [
    ['Matches', 'Minutes', 'Goals', 'Assists', 'Pass Acc %', 'Distance/90'],
    [f"{all_metrics['Total Matches']}", 
     f"{all_metrics['Total Minutes']:.0f}",
     f"{all_metrics['Goals']}",
     f"{all_metrics['Assists']}",
     f"{all_metrics['Pass Accuracy %']:.1f}%",
     f"{all_metrics['Distance per 90 (km)']:.2f} km"]
]

fig.add_trace(go.Table(
    header=dict(values=key_stats[0],
                fill_color='#00A86B',
                align='center',
                font=dict(color='white', size=14)),
    cells=dict(values=key_stats[1],
               fill_color='lavender',
               align='center',
               font=dict(size=16),
               height=30)
), row=2, col=1)

# 5. Goals & Assists Timeline
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Goals'].cumsum(),
    name='Goals',
    mode='lines+markers',
    line=dict(color='#00A86B', width=2)
), row=3, col=1)

fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Assists'].cumsum(),
    name='Assists',
    mode='lines+markers',
    line=dict(color='#FFB81C', width=2)
), row=3, col=1)

# 6. Position Distribution
position_counts = df['Position'].value_counts()
fig.add_trace(go.Bar(
    x=position_counts.values,
    y=position_counts.index,
    orientation='h',
    marker_color='#00A86B',
    showlegend=False
), row=3, col=3)

# Update layout
fig.update_layout(
    height=1200,
    title_text=f"<b>{player_display_name.upper()} - EXECUTIVE SUMMARY</b>",
    title_font_size=24,
    title_x=0.5,
    showlegend=True,
    legend=dict(x=0.55, y=0.35)
)

fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_yaxes(title_text="Cumulative", row=3, col=1)
fig.update_yaxes(title_text="Rating (0-10)", row=1, col=2, range=[0, 10])

fig.show()

# Print recommendation
print("\n" + "="*60)
print("🎯 RECOMMENDATION: ⚠️  CONDITIONAL BUY")
print("💰 Market Value: $6,500,000 (OVERVALUED)")
print("⚠️  Risk Level: MEDIUM")
print("📈 Best Fit: Counter-attacking teams needing pace")
print("="*60)



🎯 RECOMMENDATION: ⚠️  CONDITIONAL BUY
💰 Market Value: $6,500,000 (OVERVALUED)
⚠️  Risk Level: MEDIUM
📈 Best Fit: Counter-attacking teams needing pace



## RECOMMENDATION: BUY
## Market Value: $5,000,000
## Risk Level: LOW
## Best Fit: Possession-based teams needing creative width


---

# PAGE 2: TECHNICAL EVALUATION

## Rating: 7.5/10

### Analysis of passing, shooting, dribbling, and ball control

In [35]:
# Create technical evaluation dashboard
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Technical Rating', 'Pass Accuracy', 'Dribble Success',
                    'Shot Accuracy', 'Offensive Output (per 90)', 'Pass Accuracy Trend',
                    'Shooting Efficiency (xG vs Goals)', 'Key Technical Stats', 'Pass Type Distribution'),
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'bar'}, {'type': 'scatter'}],
           [{'type': 'scatter'}, {'type': 'table'}, {'type': 'bar'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Row 1: Gauges
# Technical Rating
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Technical Rating'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#00A86B"},
           'steps': [{'range': [0, 6.5], 'color': "#FFE5E5"},
                     {'range': [6.5, 7.5], 'color': "#FFF5E5"},
                     {'range': [7.5, 10], 'color': "#E5FFE5"}]}
), row=1, col=1)

# Pass Accuracy
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Pass Accuracy %'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': "#00A86B"},
           'threshold': {'line': {'color': "red", 'width': 4}, 'value': 85}}
), row=1, col=2)

# Dribble Success
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Dribble Success %'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': "#FFB81C"},
           'threshold': {'line': {'color': "red", 'width': 4}, 'value': 65}}
), row=1, col=3)

# Shot Accuracy
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Shot Accuracy %'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': "#FFB81C"},
           'threshold': {'line': {'color': "red", 'width': 4}, 'value': 50}}
), row=2, col=1)

# Offensive Output per 90
metrics_names = ['Goals/90', 'Assists/90', 'Shots/90', 'KeyPasses/90']
metrics_values = [
    all_metrics['Goals per 90'],
    all_metrics['Assists per 90'],
    all_metrics['Shots per 90'],
    all_metrics['Key Passes per 90']
]

fig.add_trace(go.Bar(
    x=metrics_names,
    y=metrics_values,
    marker_color=['#D32F2F', '#00A86B', '#FFB81C', '#00A86B'],
    text=[f'{v:.2f}' for v in metrics_values],
    textposition='outside',
    showlegend=False
), row=2, col=2)

# Pass Accuracy Trend
fig.add_trace(go.Scatter(
    x=df['Matchday'],
    y=df['Pass_Accuracy'],
    mode='lines+markers',
    line=dict(color='#00A86B', width=2),
    marker=dict(size=8),
    name='Pass Accuracy'
), row=2, col=3)

# Add average line as a scatter trace - FIXED VERSION
avg_pass_acc = all_metrics['Pass Accuracy %']
fig.add_trace(go.Scatter(
    x=[df['Matchday'].min(), df['Matchday'].max()],
    y=[avg_pass_acc, avg_pass_acc],
    mode='lines',
    line=dict(color='red', width=2, dash='dash'),
    name=f'Avg: {avg_pass_acc:.1f}%'
), row=2, col=3)

# xG vs Goals scatter
fig.add_trace(go.Scatter(
    x=df['xG'],
    y=df['Goals'],
    mode='markers',
    marker=dict(size=df['Shots_Total']*5, color='#00A86B', 
                line=dict(width=1, color='white')),
    text=df['Opponent'],
    hovertemplate='<b>%{text}</b><br>xG: %{x:.2f}<br>Goals: %{y}<extra></extra>',
    showlegend=False
), row=3, col=1)

# Add diagonal line (perfect xG = Goals)
max_xg = df['xG'].max()
fig.add_trace(go.Scatter(
    x=[0, max_xg], y=[0, max_xg],
    mode='lines',
    line=dict(dash='dash', color='red'),
    name='Perfect Conversion',
    showlegend=False
), row=3, col=1)

# Technical Stats Table
tech_stats = pd.DataFrame({
    'Metric': ['Total Passes', 'Pass Acc %', 'Total Shots', 'Shot Acc %', 
               'Total Dribbles', 'Dribble Success %'],
    'Value': [
        f"{df['Passes_Attempted'].sum()}",
        f"{all_metrics['Pass Accuracy %']:.1f}%",
        f"{all_metrics['Shots per 90']:.1f} per 90",
        f"{all_metrics['Shot Accuracy %']:.1f}%",
        f"{df['Dribbles_Attempted'].sum()}",
        f"{all_metrics['Dribble Success %']:.1f}%"
    ]
})

fig.add_trace(go.Table(
    header=dict(values=list(tech_stats.columns),
                fill_color='#00A86B',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[tech_stats.Metric, tech_stats.Value],
               fill_color='lavender',
               align='left')
), row=3, col=2)

# Pass Distribution
pass_types = ['Short', 'Medium', 'Long']
pass_counts = [
    df['Short_Passes'].sum(),
    df['Medium_Passes'].sum(),
    df['Long_Passes'].sum()
]

fig.add_trace(go.Bar(
    x=pass_types,
    y=pass_counts,
    marker_color=['#00A86B', '#FFB81C', '#D32F2F'],
    text=pass_counts,
    textposition='outside',
    showlegend=False
), row=3, col=3)

# Update layout
fig.update_layout(
    height=1200,
    title_text=f"<b>TECHNICAL EVALUATION - Rating: {all_metrics['Technical Rating']:.1f}/10</b>",
    title_font_size=22,
    title_x=0.5
)

fig.update_xaxes(title_text="Matchday", row=2, col=3)
fig.update_yaxes(title_text="Pass Accuracy %", row=2, col=3, range=[70, 100])
fig.update_xaxes(title_text="Expected Goals (xG)", row=3, col=1)
fig.update_yaxes(title_text="Actual Goals", row=3, col=1)
fig.update_xaxes(title_text="Pass Type", row=3, col=3)
fig.update_yaxes(title_text="Total Passes", row=3, col=3)

fig.show()



In [36]:


# Print analysis
print("\n" + "="*70)
print(f"📊 TECHNICAL ANALYSIS - OSMAN BUKARI - RATING: 6.6/10")
print("="*70)
print("\n✅ STRENGTHS:")
print(f"  • Elite passer: 77.6% accuracy (MLS avg: 78%)")
print(f"  • Good dribbler: 55.4% success rate")
print(f"  • Creative: 2.0 key passes per 90")
print(f"  • Quality first touch: 7.4/10 rating")
print("\n⚠️  WEAKNESSES:")
print(f"  • Limited goalscorer: 0.18 per 90")
print(f"  • Shot accuracy needs improvement: 42.9%")
print("  • Underperforms xG slightly")
print("\n💡 VERDICT:")
print("  Strong technical foundation for a winger. Best asset is pace combined with")
print("  decent dribbling. Needs to improve finishing and shot placement to reach")
print("  elite level as a goal-scoring threat.")
print("="*70)


📊 TECHNICAL ANALYSIS - OSMAN BUKARI - RATING: 6.6/10

✅ STRENGTHS:
  • Elite passer: 77.6% accuracy (MLS avg: 78%)
  • Good dribbler: 55.4% success rate
  • Creative: 2.0 key passes per 90
  • Quality first touch: 7.4/10 rating

⚠️  WEAKNESSES:
  • Limited goalscorer: 0.18 per 90
  • Shot accuracy needs improvement: 42.9%
  • Underperforms xG slightly

💡 VERDICT:
  Strong technical foundation for a winger. Best asset is pace combined with
  decent dribbling. Needs to improve finishing and shot placement to reach
  elite level as a goal-scoring threat.



TECHNICAL ANALYSIS - DAVID DA COSTA - RATING: 7.1/10

STRENGTHS:
  • Elite passer: 85.6% accuracy (MLS avg: 78%)
  • Good dribbler: 66.3% success rate
  • Creative: 2.3 key passes per 90
  • Quality first touch: 7.3/10 rating

WEAKNESSES:
  • Limited goalscorer: 0.15 per 90
  • Shot accuracy needs improvement: 42.8%
  • Underperforms xG slightly

VERDICT:
  Strong technical foundation. Best asset is passing range and
  creativity. Needs to improve finishing to reach elite level.

---

# PAGE 3: PHYSICAL EVALUATION

## Rating: 7.0/10

### Analysis of endurance, speed, strength, and duels

In [37]:
# Create physical evaluation dashboard
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Physical Rating', 'Distance per 90', 'Ground Duel Success',
                    'Aerial Success', 'Distance Covered Over Season', 'Sprint Activity',
                    'Duels Won vs Lost', 'Physical Stats Summary', 'Stamina Indicator'),
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'bar'}, {'type': 'table'}, {'type': 'scatter'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Row 1: Gauges
# Physical Rating
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Physical Rating'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#FFB81C"},
           'steps': [{'range': [0, 6.5], 'color': "#FFE5E5"},
                     {'range': [6.5, 7.5], 'color': "#FFF5E5"},
                     {'range': [7.5, 10], 'color': "#E5FFE5"}]}
), row=1, col=1)

# Distance per 90
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Distance per 90 (km)'],
    title={'text': ""},
    gauge={'axis': {'range': [8, 12]},
           'bar': {'color': "#00A86B"},
           'threshold': {'line': {'color': "red", 'width': 4}, 'value': 10.5}}
), row=1, col=2)

# Ground Duel Success
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Duel Success %'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': "#FFB81C"},
           'threshold': {'line': {'color': "red", 'width': 4}, 'value': 52}}
), row=1, col=3)

# Aerial Success
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Aerial Success %'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 100]},
           'bar': {'color': "#D32F2F"},
           'threshold': {'line': {'color': "red", 'width': 4}, 'value': 42}}
), row=2, col=1)

# Distance Over Season
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Distance_Covered_KM'],
    mode='lines+markers',
    line=dict(color='#00A86B', width=2),
    marker=dict(size=8),
    name='Distance',
    showlegend=False
), row=2, col=2)

# Add trend line
from scipy import stats
x_numeric = np.arange(len(df))
slope, intercept, r_value, p_value, std_err = stats.linregress(x_numeric, df['Distance_Covered_KM'])
trend_line = slope * x_numeric + intercept

fig.add_trace(go.Scatter(
    x=df['Date'],
    y=trend_line,
    mode='lines',
    line=dict(dash='dash', color='red'),
    name='Trend',
    showlegend=False
), row=2, col=2)

# Sprint Activity
fig.add_trace(go.Scatter(
    x=df['Matchday'],
    y=df['Sprints'],
    mode='lines',
    fill='tozeroy',
    line=dict(color='#00A86B'),
    showlegend=False
), row=2, col=3)

# Duels Won vs Lost
duels_won = df['Duels_Won'].sum()
duels_lost = df['Duels_Total'].sum() - duels_won

fig.add_trace(go.Bar(
    x=['Won', 'Lost'],
    y=[duels_won, duels_lost],
    marker_color=['#00A86B', '#D32F2F'],
    text=[duels_won, duels_lost],
    textposition='outside',
    showlegend=False
), row=3, col=1)

# Physical Stats Table
phys_stats = pd.DataFrame({
    'Metric': ['Avg Distance/90', 'Top Speed', 'HI Runs/90', 'Sprints/90', 'Strength Rating'],
    'Value': [
        f"{all_metrics['Distance per 90 (km)']:.2f} km",
        f"{all_metrics['Top Speed (km/h)']:.1f} km/h",
        f"{all_metrics['HI Runs per 90']:.1f}",
        f"{all_metrics['Sprints per 90']:.1f}",
        f"{metrics.avg_strength_rating():.1f}/10"
    ]
})

fig.add_trace(go.Table(
    header=dict(values=list(phys_stats.columns),
                fill_color='#00A86B',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[phys_stats.Metric, phys_stats.Value],
               fill_color='lavender',
               align='left')
), row=3, col=2)

# Stamina Indicator (2nd half vs 1st half)
fig.add_trace(go.Scatter(
    x=df['Matchday'],
    y=df['Stamina_Indicator'],
    mode='lines+markers',
    line=dict(color='#00A86B', width=2),
    marker=dict(size=8),
    name='Stamina',
    showlegend=False
), row=3, col=3)

# Add reference line at 0 - FIXED VERSION
fig.add_trace(go.Scatter(
    x=[df['Matchday'].min(), df['Matchday'].max()],
    y=[0, 0],
    mode='lines',
    line=dict(dash='dash', color='gray'),
    name='Baseline',
    showlegend=False
), row=3, col=3)

# Update layout
fig.update_layout(
    height=1200,
    title_text=f"<b>PHYSICAL EVALUATION - Rating: {all_metrics['Physical Rating']:.1f}/10</b>",
    title_font_size=22,
    title_x=0.5
)

fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_yaxes(title_text="Distance (km)", row=2, col=2)
fig.update_xaxes(title_text="Matchday", row=2, col=3)
fig.update_yaxes(title_text="Sprints", row=2, col=3)
fig.update_yaxes(title_text="Total Duels", row=3, col=1)
fig.update_xaxes(title_text="Matchday", row=3, col=3)
fig.update_yaxes(title_text="Stamina Indicator", row=3, col=3)

fig.show()


In [38]:


# Print analysis
print("\n" + "="*70)
print(f"📊 PHYSICAL ANALYSIS - OSMAN BUKARI - RATING: 7.4/10")
print("="*70)
print("\n✅ STRENGTHS:")
print(f"  • Elite pace: 34.3 km/h top speed - exceptional!")
print(f"  • Excellent endurance: 10.15 km per 90")
print(f"  • High work rate: 66.5 HI runs per 90")
print("  • Maintains performance in 2nd half (positive stamina indicator)")
print("\n⚠️  WEAKNESSES:")
print(f"  • Weak aerially: 29.7% success (MLS avg: 42%)")
print(f"  • Can be out-muscled: 5.5/10 strength")
print(f"  • Average ground duel success: 46.6%")
print("\n💡 VERDICT:")
print("  Outstanding physical profile for a winger. Elite pace is the standout")
print("  attribute - 34.3 km/h puts him in top tier. Covers")
print("  ground well. Physical presence is a weakness - vulnerable in duels.")
print("="*70)


📊 PHYSICAL ANALYSIS - OSMAN BUKARI - RATING: 7.4/10

✅ STRENGTHS:
  • Elite pace: 34.3 km/h top speed - exceptional!
  • Excellent endurance: 10.15 km per 90
  • High work rate: 66.5 HI runs per 90
  • Maintains performance in 2nd half (positive stamina indicator)

⚠️  WEAKNESSES:
  • Weak aerially: 29.7% success (MLS avg: 42%)
  • Can be out-muscled: 5.5/10 strength
  • Average ground duel success: 46.6%

💡 VERDICT:
  Outstanding physical profile for a winger. Elite pace is the standout
  attribute - 34.3 km/h puts him in top tier. Covers
  ground well. Physical presence is a weakness - vulnerable in duels.



📊 PHYSICAL ANALYSIS - DAVID DA COSTA - RATING: 7.2/10


✅ STRENGTHS:
  • Excellent endurance: 10.52 km per 90
  • High work rate: 54.6 HI runs per 90
  • Maintains performance in 2nd half (positive stamina indicator)
  • Good acceleration: 7.5/10 rating

⚠️  WEAKNESSES:
  • Weak aerially: 35.6% success (MLS avg: 42%)
  • Can be out-muscled: 6.0/10 strength
  • Average ground duel success: 52.2%

💡 VERDICT:
  Good athlete with standout endurance. Covers ground well and
  maintains intensity. Physical presence is a weakness - can be
  dominated by stronger opponents.

---

# PAGE 4: TACTICAL EVALUATION

## Rating: 6.8/10

### Analysis of positioning, decision-making, and tactical intelligence

In [39]:
# Create tactical evaluation dashboard
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Tactical Rating', 'Positioning', 'Decision Making',
                    'Def Work Rate', 'Defensive Actions', 'Defensive Actions Over Time',
                    'Performance by Position', 'Tactical Stats', 'Pressing Effectiveness'),
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'bar'}, {'type': 'scatter'}],
           [{'type': 'bar'}, {'type': 'table'}, {'type': 'bar'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Row 1: Tactical Ratings
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Tactical Rating'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#FFB81C"},
           'steps': [{'range': [0, 6.5], 'color': "#FFE5E5"},
                     {'range': [6.5, 7.5], 'color': "#FFF5E5"},
                     {'range': [7.5, 10], 'color': "#E5FFE5"}]}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=metrics.avg_positioning_rating(),  # FIXED: Use method directly
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#FFB81C"}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=metrics.avg_decision_making(),  # FIXED: Use method directly
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#FFB81C"}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=metrics.avg_defensive_work_rate(),  # FIXED: Use method directly
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#00A86B"}}
), row=2, col=1)

# Defensive Actions Breakdown
def_actions = ['Tackles', 'Interceptions', 'Clearances', 'Blocks']
def_values = [
    df['Tackles_Won'].sum(),
    df['Interceptions'].sum(),
    df['Clearances'].sum(),
    df['Blocks'].sum()
]

fig.add_trace(go.Bar(
    x=def_actions,
    y=def_values,
    marker_color=['#00A86B', '#FFB81C', '#D32F2F', '#00A86B'],
    text=def_values,
    textposition='outside',
    showlegend=False
), row=2, col=2)

# Defensive Actions Over Time
df['Total_Def_Actions'] = df['Tackles_Won'] + df['Interceptions'] + df['Clearances'] + df['Blocks']

fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Total_Def_Actions'],
    mode='lines+markers',
    line=dict(color='#00A86B', width=2),
    marker=dict(size=8),
    showlegend=False
), row=2, col=3)

# Performance by Position
position_performance = df.groupby('Position')['Performance_Rating'].mean().sort_values(ascending=False)

fig.add_trace(go.Bar(
    y=position_performance.index,
    x=position_performance.values,
    orientation='h',
    marker_color='#00A86B',
    text=[f'{v:.1f}' for v in position_performance.values],
    textposition='outside',
    showlegend=False
), row=3, col=1)

# Tactical Stats Table
tact_stats = pd.DataFrame({
    'Metric': ['Tackles/90', 'Interceptions/90', 'Def Actions/90', 
               'Pressures/90', 'Pressure Success %'],
    'Value': [
        f"{all_metrics['Tackles per 90']:.2f}",
        f"{all_metrics['Interceptions per 90']:.2f}",
        f"{all_metrics['Defensive Actions per 90']:.2f}",
        f"{all_metrics['Pressures per 90']:.1f}",
        f"{all_metrics['Pressure Success %']:.1f}%"
    ]
})

fig.add_trace(go.Table(
    header=dict(values=list(tact_stats.columns),
                fill_color='#00A86B',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[tact_stats.Metric, tact_stats.Value],
               fill_color='lavender',
               align='left')
), row=3, col=2)

# Pressing Effectiveness
df['Pressure_Success_Rate'] = (df['Successful_Pressures'] / df['Pressures_Applied'] * 100).fillna(0)

fig.add_trace(go.Bar(
    x=df['Matchday'],
    y=df['Pressure_Success_Rate'],
    marker_color='#FFB81C',
    showlegend=False
), row=3, col=3)

# Add average line - FIXED VERSION
avg_pressure_success = all_metrics['Pressure Success %']
fig.add_trace(go.Scatter(
    x=[df['Matchday'].min(), df['Matchday'].max()],
    y=[avg_pressure_success, avg_pressure_success],
    mode='lines',
    line=dict(dash='dash', color='red'),
    name=f'Avg: {avg_pressure_success:.1f}%',
    showlegend=False
), row=3, col=3)

# Update layout
fig.update_layout(
    height=1200,
    title_text=f"<b>TACTICAL EVALUATION - Rating: {all_metrics['Tactical Rating']:.1f}/10</b>",
    title_font_size=22,
    title_x=0.5
)

fig.update_xaxes(title_text="Date", row=2, col=3)
fig.update_yaxes(title_text="Defensive Actions", row=2, col=3)
fig.update_xaxes(title_text="Avg Performance Rating", row=3, col=1)
fig.update_xaxes(title_text="Matchday", row=3, col=3)
fig.update_yaxes(title_text="Success Rate %", row=3, col=3)

fig.show()



In [40]:


# Print analysis
print("\n" + "="*70)
print(f"📊 TACTICAL ANALYSIS - OSMAN BUKARI - RATING: 4.4/10")
print("="*70)
print("\n✅ STRENGTHS:")
print(f"  • Versatile - effective in 4 positions")
print(f"  • Good positioning awareness: 5.9/10")
print(f"  • High work rate defensively: 5.6/10")
print("  • Makes intelligent off-ball runs")
print("\n⚠️  WEAKNESSES:")
print(f"  • Decision-making can improve: 6.2/10")
print(f"  • Limited defensive output: 0.05 tackles/90")
print(f"  • Pressing could be more effective: 27.0%")
print("\n💡 VERDICT:")
print("  Tactically limited for a winger. Positioning is decent but decision-making")
print("  in final third needs significant work. Defensive contribution is minimal.")
print("  Best used as a direct, pace-focused outlet rather than tactical playmaker.")
print("="*70)


📊 TACTICAL ANALYSIS - OSMAN BUKARI - RATING: 4.4/10

✅ STRENGTHS:
  • Versatile - effective in 4 positions
  • Good positioning awareness: 5.9/10
  • High work rate defensively: 5.6/10
  • Makes intelligent off-ball runs

⚠️  WEAKNESSES:
  • Decision-making can improve: 6.2/10
  • Limited defensive output: 0.05 tackles/90
  • Pressing could be more effective: 27.0%

💡 VERDICT:
  Tactically limited for a winger. Positioning is decent but decision-making
  in final third needs significant work. Defensive contribution is minimal.
  Best used as a direct, pace-focused outlet rather than tactical playmaker.



======================================================================
📊 TACTICAL ANALYSIS - DAVID DA COSTA - RATING: 7.3/10
======================================================================

✅ STRENGTHS:
  • Versatile - effective in 5 positions
  • Good positioning awareness: 6.7/10
  • High work rate defensively: 7.0/10
  • Makes intelligent off-ball runs

⚠️  WEAKNESSES:
  • Decision-making can improve: 7.0/10
  • Limited defensive output: 0.74 tackles/90
  • Pressing could be more effective: 32.7%

💡 VERDICT:
  Tactically sound player who understands his role. Best in LM/LW
  positions. Needs to improve defensive contribution and decision-
  making in final third.

---

# PAGE 5: PSYCHOLOGICAL EVALUATION

## Rating: 7.5/10

### Analysis of consistency, discipline, confidence, and mental strength

In [41]:
# Create psychological evaluation dashboard
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=('Psychological Rating', 'Performance Rating', 'Consistency',
                    'Discipline', 'Performance Consistency Over Season', 'Pressure Performance',
                    'Performance by Result', 'Confidence Trajectory', 'Disciplinary Record'),
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}, {'type': 'indicator'}],
           [{'type': 'indicator'}, {'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'bar'}, {'type': 'scatter'}, {'type': 'funnel'}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Row 1: Psychological Ratings
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Psychological Rating'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#00A86B"},
           'steps': [{'range': [0, 6.5], 'color': "#FFE5E5"},
                     {'range': [6.5, 7.5], 'color': "#FFF5E5"},
                     {'range': [7.5, 10], 'color': "#E5FFE5"}]}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Avg Performance Rating'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#00A86B"}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Performance Consistency'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#00A86B"}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=all_metrics['Discipline Rating'],
    title={'text': ""},
    gauge={'axis': {'range': [0, 10]},
           'bar': {'color': "#00A86B"}}
), row=2, col=1)

# Performance Consistency Over Season
fig.add_trace(go.Scatter(
    x=df['Matchday'],
    y=df['Performance_Rating'],
    mode='lines+markers',
    line=dict(color='#00A86B', width=2),
    marker=dict(size=8),
    name='Performance',
    showlegend=False
), row=2, col=2)

# Add average line - FIXED VERSION
avg_perf = all_metrics['Avg Performance Rating']
fig.add_trace(go.Scatter(
    x=[df['Matchday'].min(), df['Matchday'].max()],
    y=[avg_perf, avg_perf],
    mode='lines',
    line=dict(dash='dash', color='red'),
    name=f'Avg: {avg_perf:.1f}',
    showlegend=False
), row=2, col=2)

# Pressure Performance (Big games vs normal games)
fig.add_trace(go.Scatter(
    x=df['Matchday'],
    y=df['Pressure_Performance_Rating'],
    mode='markers',
    marker=dict(
        size=12,
        color=df['Is_Big_Game'],
        colorscale=[[0, '#FFB81C'], [1, '#00A86B']],
        showscale=False
    ),
    text=['Big Game' if x == 1 else 'Regular' for x in df['Is_Big_Game']],
    hovertemplate='<b>Matchday %{x}</b><br>Performance: %{y:.1f}<br>%{text}<extra></extra>',
    showlegend=False
), row=2, col=3)

# Performance by Result
result_performance = df.groupby('Result')['Performance_Rating'].mean()
result_colors = {'W': '#00A86B', 'D': '#FFB81C', 'L': '#D32F2F'}
colors_list = [result_colors[r] for r in result_performance.index]

fig.add_trace(go.Bar(
    x=['Win', 'Draw', 'Loss'],
    y=[result_performance.get('W', 0), result_performance.get('D', 0), result_performance.get('L', 0)],
    marker_color=colors_list,
    text=[f'{v:.1f}' for v in [result_performance.get('W', 0), 
                                result_performance.get('D', 0), 
                                result_performance.get('L', 0)]],
    textposition='outside',
    showlegend=False
), row=3, col=1)

# Confidence Trajectory
fig.add_trace(go.Scatter(
    x=df['Matchday'],
    y=df['Confidence_Level'],
    mode='lines+markers',
    line=dict(color='#00A86B', width=2),
    marker=dict(size=8),
    fill='tozeroy',
    showlegend=False
), row=3, col=2)

# Add trend line
slope_conf, intercept_conf, r_value, p_value, std_err = stats.linregress(df['Matchday'], df['Confidence_Level'])
trend_line_conf = slope_conf * df['Matchday'] + intercept_conf

fig.add_trace(go.Scatter(
    x=df['Matchday'],
    y=trend_line_conf,
    mode='lines',
    line=dict(dash='dash', color='red'),
    name='Trend',
    showlegend=False
), row=3, col=2)

# Disciplinary Record Funnel
matches_with_fouls = (df['Fouls_Committed'] > 0).sum()
yellow_cards = all_metrics['Yellow Cards']
red_cards = all_metrics['Red Cards']

fig.add_trace(go.Funnel(
    y=['Total Matches', 'Matches with Fouls', 'Yellow Cards', 'Red Cards'],
    x=[all_metrics['Total Matches'], matches_with_fouls, yellow_cards, red_cards],
    marker=dict(color=['#00A86B', '#FFB81C', '#FFA500', '#D32F2F']),
    textinfo="value+percent initial"
), row=3, col=3)

# Update layout
fig.update_layout(
    height=1200,
    title_text=f"<b>PSYCHOLOGICAL EVALUATION - Rating: {all_metrics['Psychological Rating']:.1f}/10</b>",
    title_font_size=22,
    title_x=0.5
)

fig.update_xaxes(title_text="Matchday", row=2, col=2)
fig.update_yaxes(title_text="Performance Rating", row=2, col=2, range=[5, 10])
fig.update_xaxes(title_text="Matchday", row=2, col=3)
fig.update_yaxes(title_text="Pressure Performance", row=2, col=3)
fig.update_yaxes(title_text="Avg Performance", row=3, col=1, range=[0, 10])
fig.update_xaxes(title_text="Matchday", row=3, col=2)
fig.update_yaxes(title_text="Confidence Level", row=3, col=2, range=[5, 10])

fig.show()



In [42]:


# Print analysis
print("\n" + "="*70)
print(f"📊 PSYCHOLOGICAL ANALYSIS - OSMAN BUKARI - RATING: 5.5/10")
print("="*70)
print("\n✅ STRENGTHS:")
print(f"  • Good discipline: 5.0/10 rating")
print(f"  • Consistent performer: 6.7/10 avg")
print(f"  • Handles pressure well: 6.4/10")
print(f"  • Strong composure: 7.2/10")
print(f"  • Growing confidence: 0.071 increase per match")
print("\n⚠️  WEAKNESSES:")
print("  • Performance variance needs reduction")
print("  • Occasional frustration when isolated")
print("  • Confidence dependent on early touches")
print("\n💡 VERDICT:")
print("  Mentally solid player with professional attitude. Discipline record is")
print("  good. Consistency could improve. Confidence grows with game time.")
print("  Best when given freedom to express pace-based game.")
print("="*70)


📊 PSYCHOLOGICAL ANALYSIS - OSMAN BUKARI - RATING: 5.5/10

✅ STRENGTHS:
  • Good discipline: 5.0/10 rating
  • Consistent performer: 6.7/10 avg
  • Handles pressure well: 6.4/10
  • Strong composure: 7.2/10
  • Growing confidence: 0.071 increase per match

⚠️  WEAKNESSES:
  • Performance variance needs reduction
  • Occasional frustration when isolated
  • Confidence dependent on early touches

💡 VERDICT:
  Mentally solid player with professional attitude. Discipline record is
  good. Consistency could improve. Confidence grows with game time.
  Best when given freedom to express pace-based game.



======================================================================
📊 PSYCHOLOGICAL ANALYSIS - DAVID DA COSTA - RATING: 7.6/10
======================================================================

✅ STRENGTHS:
  • Excellent discipline: 7.3/10 rating
  • Consistent performer: 7.3/10 avg
  • Handles pressure well: 7.1/10
  • Strong composure: 7.6/10
  • Growing confidence: -0.017 increase per match

⚠️  WEAKNESSES:
  • Performance variance needs reduction
  • Occasional frustration when losing
  • Mental fatigue visible late season

💡 VERDICT:
  Mentally strong player with professional attitude. Excellent
  disciplinary record. Consistency is good but can improve. Handles
  big games well. Confidence growing - positive trajectory.

---

# PAGE 6: MATCH-BY-MATCH BREAKDOWN


In [43]:
# Create comprehensive match-by-match table
match_breakdown = df[[
    'Date', 'Opponent', 'Result', 'Score', 'Position', 'Minutes',
    'Goals', 'Assists', 'Pass_Accuracy', 'Distance_Covered_KM',
    'Total_Def_Actions', 'Performance_Rating'
]].copy()

# Round numeric columns
match_breakdown['Pass_Accuracy'] = match_breakdown['Pass_Accuracy'].round(1)
match_breakdown['Distance_Covered_KM'] = match_breakdown['Distance_Covered_KM'].round(2)
match_breakdown['Performance_Rating'] = match_breakdown['Performance_Rating'].round(1)

# Create figure
fig = go.Figure()

# Create table with conditional formatting
def get_cell_color(col_name, values):
    if col_name == 'Result':
        return ['#E5FFE5' if v == 'W' else '#FFF5E5' if v == 'D' else '#FFE5E5' for v in values]
    elif col_name in ['Goals', 'Assists']:
        return ['#E5FFE5' if v > 0 else 'white' for v in values]
    elif col_name == 'Pass_Accuracy':
        return ['#E5FFE5' if v >= 85 else '#FFE5E5' if v < 75 else 'white' for v in values]
    elif col_name == 'Performance_Rating':
        return ['#E5FFE5' if v >= 7.5 else '#FFE5E5' if v < 6.0 else 'white' for v in values]
    else:
        return 'white'

cell_colors = [get_cell_color(col, match_breakdown[col]) 
               for col in match_breakdown.columns]

fig.add_trace(go.Table(
    header=dict(
        values=['<b>' + col + '</b>' for col in match_breakdown.columns],
        fill_color='#00A86B',
        align='left',
        font=dict(color='white', size=12, family="Arial")
    ),
    cells=dict(
        values=[match_breakdown[col] for col in match_breakdown.columns],
        fill_color=cell_colors,
        align='left',
        font=dict(size=11),
        height=25
    )
))

fig.update_layout(
    title="<b>MATCH-BY-MATCH BREAKDOWN</b>",
    title_font_size=22,
    title_x=0.5,
    height=800
)

fig.show()

# Summary statistics by position
print("\n" + "="*70)
print("📊 PERFORMANCE BY POSITION")
print("="*70)

position_stats = df.groupby('Position').agg({
    'Match_ID': 'count',
    'Minutes': 'mean',
    'Goals': 'sum',
    'Assists': 'sum',
    'Pass_Accuracy': 'mean',
    'Performance_Rating': 'mean'
}).round(2)

position_stats.columns = ['Matches', 'Avg Mins', 'Goals', 'Assists', 'Pass Acc %', 'Avg Rating']


📊 PERFORMANCE BY POSITION


In [44]:
print(position_stats.to_string())
print("="*70)

          Matches  Avg Mins  Goals  Assists  Pass Acc %  Avg Rating
Position                                                           
LM              5     82.60      1        0       74.18        6.50
LW              9     74.67      2        3       78.93        7.34
RM              6     77.33      1        1       76.78        6.28
RW              5     88.80      0        3       79.44        6.40


======================================================================
📊 PERFORMANCE BY POSITION
======================================================================
          Matches  Avg Mins  Goals  Assists  Pass Acc %  Avg Rating
Position                                                           
CAM             4     62.00      0        1       87.10        7.15
LM              7     85.86      0        2       85.57        7.34
LW              2     88.50      0        2       86.70        7.00
RM              6     63.67      0        0       86.75        7.37
RW              6     69.33      3        2       82.95        7.32
======================================================================

---

# PAGE 7: FINAL RECOMMENDATION

## Overall verdict and transfer recommendation

In [45]:
# Create final recommendation dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Overall Rating', 'Four Pillar Comparison', 
                    'Key Metrics vs Benchmarks', 'Strengths vs Weaknesses'),
    specs=[[{'type': 'indicator'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'bar'}]],
    row_heights=[0.5, 0.5],
    vertical_spacing=0.15,
    horizontal_spacing=0.15
)

# Overall Rating (Large)
fig.add_trace(go.Indicator(
    mode="gauge+number+delta",
    value=all_metrics['Overall Rating'],
    title={'text': "<b>Overall Rating</b>", 'font': {'size': 28}},
    delta={'reference': 7.5, 'font': {'size': 20}},
    gauge={
        'axis': {'range': [None, 10], 'tickwidth': 2},
        'bar': {'color': "#00A86B", 'thickness': 0.8},
        'bgcolor': "white",
        'borderwidth': 3,
        'bordercolor': "gray",
        'steps': [
            {'range': [0, 6.5], 'color': '#FFE5E5'},
            {'range': [6.5, 7.5], 'color': '#FFF5E5'},
            {'range': [7.5, 10], 'color': '#E5FFE5'}],
        'threshold': {
            'line': {'color': "red", 'width': 6},
            'thickness': 0.85,
            'value': 7.5}
    },
    number={'font': {'size': 60}}
), row=1, col=1)

# Four Pillar Horizontal Bar
pillars = ['Technical', 'Physical', 'Tactical', 'Psychological']
ratings = [
    all_metrics['Technical Rating'],
    all_metrics['Physical Rating'],
    all_metrics['Tactical Rating'],
    all_metrics['Psychological Rating']
]
colors = ['#00A86B' if r >= 7.5 else '#FFB81C' if r >= 6.5 else '#D32F2F' for r in ratings]

fig.add_trace(go.Bar(
    y=pillars,
    x=ratings,
    orientation='h',
    marker_color=colors,
    text=[f'{r:.1f}/10' for r in ratings],
    textposition='outside',
    showlegend=False
), row=1, col=2)

# Key Metrics vs Benchmarks
metrics_comparison = pd.DataFrame({
    'Metric': ['Pass Acc %', 'Goals/90', 'Assists/90', 'Distance/90', 'Discipline'],
    '{player_display_name}': [
        all_metrics['Pass Accuracy %'],
        all_metrics['Goals per 90'] * 100,  # Scale for visibility
        all_metrics['Assists per 90'] * 100,
        all_metrics['Distance per 90 (km)'] * 10,
        all_metrics['Discipline Rating'] * 10
    ],
    'MLS Avg': [78, 35, 28, 102, 65],  # Scaled values
    'Top 10%': [88, 65, 55, 115, 92]
})

fig.add_trace(go.Bar(
    name='{player_display_name}',
    x=metrics_comparison['Metric'],
    y=metrics_comparison['{player_display_name}'],
    marker_color='#00A86B'
), row=2, col=1)

fig.add_trace(go.Bar(
    name='MLS Average',
    x=metrics_comparison['Metric'],
    y=metrics_comparison['MLS Avg'],
    marker_color='#FFB81C'
), row=2, col=1)

fig.add_trace(go.Bar(
    name='Top 10%',
    x=metrics_comparison['Metric'],
    y=metrics_comparison['Top 10%'],
    marker_color='#D32F2F'
), row=2, col=1)

# Strengths vs Weaknesses
strengths_weaknesses = pd.DataFrame({
    'Category': ['Passing', 'Discipline', 'Endurance', 'Creativity',
                 'Finishing', 'Aerial', 'Strength', 'Defensive'],
    'Rating': [9, 9, 8, 8, 5, 4, 6, 6]
})

colors_sw = ['#00A86B' if r >= 7.5 else '#D32F2F' if r < 6.5 else '#FFB81C' 
             for r in strengths_weaknesses['Rating']]

fig.add_trace(go.Bar(
    y=strengths_weaknesses['Category'],
    x=strengths_weaknesses['Rating'],
    orientation='h',
    marker_color=colors_sw,
    text=strengths_weaknesses['Rating'],
    textposition='outside',
    showlegend=False
), row=2, col=2)

# Update layout
fig.update_layout(
    height=900,
    title_text="<b>FINAL RECOMMENDATION</b>",
    title_font_size=26,
    title_x=0.5,
    showlegend=True,
    legend=dict(x=0.1, y=0.4, orientation='v')
)

fig.update_xaxes(title_text="Rating (0-10)", row=1, col=2, range=[0, 10])
fig.update_xaxes(title_text="Metric", row=2, col=1)
fig.update_yaxes(title_text="Scaled Value", row=2, col=1)
fig.update_xaxes(title_text="Rating (0-10)", row=2, col=2, range=[0, 10])

fig.show()

In [46]:


# Print Final Recommendation
print("\n" + "="*80)
print("")
print("  ╔══════════════════════════════════════════════════════════════════════╗")
print("  ║                                                                      ║")
print(f"  ║              OVERALL RATING: 6.0/10                                ║")
print("  ║                                                                      ║")
print("  ║              RECOMMENDATION: ⚠️  CONDITIONAL BUY                     ║")
print("  ║                                                                      ║")
print("  ╚══════════════════════════════════════════════════════════════════════╝")
print("")
print("="*80)
print("\n📋 PLAYER PROFILE")
print("─" * 80)
print("Speed-focused winger with elite pace. Direct style, less technical")
print("than typical creative wingers. Best as impact substitute or counter-")
print("attacking outlet.")
print("\n🎯 BEST FIT")
print("─" * 80)
print("✓ Counter-attacking teams (fast transitions)")
print("✓ Systems: 4-3-3, 4-2-3-1 (wide right)")
print("✓ Teams needing pace in behind")
print("✓ Not suited for possession-heavy systems")
print("\n💰 MARKET VALUE")
print("─" * 80)
print("Current Valuation: $6,500,000")
print("Fair Range: $5M - $7M")
print("Recommended Offer: $5M - $5.5M (overvalued)")
print("\n⚠️  RISK ASSESSMENT")
print("─" * 80)
print("• Age Risk: LOW (25, entering prime)")
print("• Injury Risk: MEDIUM (high-intensity style)")
print("• Adaptation Risk: MEDIUM (tactical limitations)")
print("• Overall Risk: MEDIUM")
print("\n✅ STRENGTHS SUMMARY")
print("─" * 80)
print(f"• Elite pace (34.3 km/h top speed) - game-changing")
print(f"• Good discipline (3 yellow in 25 games)")
print(f"• High work rate (10.15 km per 90)")
print("• Versatile positioning")
print(f"• Decent dribbler (55.4% success)")
print("\n⚠️  WEAKNESSES SUMMARY")
print("─" * 80)
print(f"• Very limited goalscorer (0.18 per 90)")
print(f"• Weak aerially (29.7% success)")
print("• Can be out-muscled physically")
print(f"• Poor decision-making (6.2/10)")
print(f"• Tactically limited (4.4/10)")
print("\n💡 FINAL VERDICT")
print("─" * 80)
print("One-dimensional winger whose primary asset is exceptional pace. At $6.5M,")
print("he's overvalued given tactical limitations and low goal output. Elite speed")
print("makes him useful as impact substitute for counter-attacking situations, but")
print("lacks the technical quality and tactical intelligence for starting role in")
print("possession-based systems.")
print("")
print("Recommendation: PASS at current valuation. Consider only if price drops to")
print("$5M and team plays direct, counter-attacking style. Better options exist")
print("for creative wing play.")
print("")
print("  ╔══════════════════════════════════════════════════════════════════════╗")
print("  ║                                                                      ║")
print("  ║              CONDITIONAL APPROVAL - PRICE DEPENDENT                  ║")
print("  ║                                                                      ║")
print("  ╚══════════════════════════════════════════════════════════════════════╝")
print("")
print("="*80)



  ╔══════════════════════════════════════════════════════════════════════╗
  ║                                                                      ║
  ║              OVERALL RATING: 6.0/10                                ║
  ║                                                                      ║
  ║              RECOMMENDATION: ⚠️  CONDITIONAL BUY                     ║
  ║                                                                      ║
  ╚══════════════════════════════════════════════════════════════════════╝


📋 PLAYER PROFILE
────────────────────────────────────────────────────────────────────────────────
Speed-focused winger with elite pace. Direct style, less technical
than typical creative wingers. Best as impact substitute or counter-
attacking outlet.

🎯 BEST FIT
────────────────────────────────────────────────────────────────────────────────
✓ Counter-attacking teams (fast transitions)
✓ Systems: 4-3-3, 4-2-3-1 (wide right)
✓ Teams needing pace in behind
✓ Not suited 

OVERALL RATING: 7.3/10
RECOMMENDATION: BUY 


PLAYER PROFILE
────────────────────────────────────────────────────────────────────────────────
Creative midfielder with excellent passing and work rate.
Versatile across multiple positions in attacking midfield.

BEST FIT
────────────────────────────────────────────────────────────────────────────────
✓ Possession-based teams (60%+ possession)
✓ Systems: 4-3-3, 4-2-3-1 (wide player)
✓ Teams needing creative ball progression
✓ Mid-table to upper-mid-table clubs

MARKET VALUE
────────────────────────────────────────────────────────────────────────────────
Current Valuation: $5,000,000
Fair Range: $4M - $6M
Recommended Offer: $4.5M - $5.5M

RISK ASSESSMENT
────────────────────────────────────────────────────────────────────────────────
• Age Risk: LOW (26, entering prime)
• Injury Risk: LOW (good history)
• Adaptation Risk: LOW (proven in MLS)
• Overall Risk: LOW

STRENGTHS SUMMARY
────────────────────────────────────────────────────────────────────────────────
• Elite passer (85.6% accuracy)
• Excellent discipline (1 yellow in 25 games)
• High work rate (10.52 km per 90)
• Versatile positioning
• Consistent performer (7.3/10 avg)

WEAKNESSES SUMMARY
────────────────────────────────────────────────────────────────────────────────
• Limited goalscorer (0.15 per 90)
• Weak aerially (35.6% success)
• Can be out-muscled physically
• Decision-making needs refinement

FINAL VERDICT
────────────────────────────────────────────────────────────────────────────────
Quality MLS-level creative midfielder who excels in ball progression
and link-up play. Best utilized in possession-dominant systems where
technical quality is prioritized over physicality. Age 26 suggests room
for growth.

At $5M valuation, represents good value for teams seeking reliable
creativity from wide areas or central midfield. Low risk profile due
to proven MLS experience.

APPROVED FOR RECRUITMENT 



**Report compiled by:** Jacob Farrington
**Date:** January 29, 2026  
**Contact:** jtsfarrington@gmail.com